In [3]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [4]:
def countyName(x):
    sp = x.split(' ')
    
    if sp[0] == '경기':
        sp[0] = '경기도'
        
    elif sp[0] == '경남':
        sp[0] = '경상남도'
        
    elif sp[0] == '경북':
        sp[0] = '경상북도'
        
    elif sp[0] == '충남':
        sp[0] = '충청남도'
        
    elif sp[0] == '충북':
        sp[0] = '충청북도'
        
    elif sp[0] == '전남':
        sp[0] = '전라남도'
        
    elif sp[0] == '전북':
        sp[0] = '전라북도'
        
    elif sp[0] == '인천':
        sp[0] = '인천광역시'
    
    elif sp[0] == '광주':
        sp[0] = '광주광역시'
    
    elif sp[0] == '대구':
        sp[0] = '대구광역시'
        
    elif sp[0] == '대전':
        sp[0] = '대전광역시'
        
    elif sp[0] == '부산':
        sp[0] = '부산광역시'
        
    elif sp[0] == '서울':
        sp[0] = '서울특별시'
        
    elif sp[0] == '울산':
        sp[0] = '울산광역시'
        
    elif sp[0] == '제주도':
        sp[0] = '제주특별자치도'
        
    elif sp[0] == '제주':
        sp[0] = '제주특별자치도'
        
    elif sp[0] == '세종':
        sp[0] = '세종특별자치시'    
        
    elif sp[0] == '세종시':
        sp[0] = '세종특별자치시'
##########################################################################        
    elif sp[1] == '한라대학로':
        sp[1] = '제주시'
    
    return " ".join(sp)

In [5]:
# def one_page_crawler(driver,brand,food_type,df):
#     html = BeautifulSoup(driver.page_source, 'html.parser')
#     for i in range(15):
#         if html.select('a.link_name')[i].text.split(' ')[0] == brand: #다른가게 크롤방지
#             df = np.append(df,food_type)
#             df = np.append(df,brand)
#             df = np.append(df, countyName(" ".join(html.select('div.addr')[i].text.replace('\n','').split(' ')[0:2])))
#     time.sleep(1.5)

In [15]:
def kakao_map_crawler(food_type, brand):

    df = np.array([])
    url = 'https://map.kakao.com'
    driver = webdriver.Chrome('./chromedriver')
    time.sleep(2)
    driver.get(url)
    time.sleep(2)
    html = BeautifulSoup(driver.page_source, 'html.parser')

    driver.find_element_by_xpath('//*[@id="search.keyword.query"]').send_keys(f'{brand}\n')
    time.sleep(1)

    driver.find_element_by_xpath('/html/body/div[10]/div/div/div').click()
    time.sleep(1)

    driver.find_element_by_xpath('//*[@id="info.main.options"]/li[2]').click()
    time.sleep(1)

    #1페이지크롤
    html = BeautifulSoup(driver.page_source, 'html.parser')
    for i in range(15):
        if html.select('a.link_name')[i].text.split(' ')[0] == brand: #다른가게 크롤방지
            df = np.append(df,food_type)
            df = np.append(df,brand)
            df = np.append(df, countyName(" ".join(html.select('div.addr')[i].text.replace('\n','').split(' ')[0:2])))
    time.sleep(1.5)

    #2~5페이지크롤
    for j in range(2,6):
        driver.find_element_by_xpath(f'//*[@id="info.search.page.no{j}"]').click()
        time.sleep(2)
        html = BeautifulSoup(driver.page_source, 'html.parser')
        for i in range(15):
            if html.select('a.link_name')[i].text.split(' ')[0] == brand: #다른가게 크롤방지
                df = np.append(df, food_type)
                df = np.append(df, brand)
                df = np.append(df, countyName(" ".join(html.select('div.addr')[i].text.replace('\n','').split(' ')[0:2])))


    #그다음 5개로 넘어가기
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="info.search.page.next"]').click() #다음 5페이지
            time.sleep(2)
            html = BeautifulSoup(driver.page_source, 'html.parser')
            for i in range(15):
                if html.select('a.link_name')[i].text.split(' ')[0] == brand: #다른가게 크롤방지
                    df = np.append(df, food_type)
                    df = np.append(df, brand)
                    df = np.append(df, countyName(" ".join(html.select('div.addr')[i].text.replace('\n','').split(' ')[0:2])))

            for k in range(2,6):
                driver.find_element_by_xpath(f'//*[@id="info.search.page.no{k}"]').click()
                time.sleep(2)
                html = BeautifulSoup(driver.page_source, 'html.parser')
                for i in range(15):
                    if html.select('a.link_name')[i].text.split(' ')[0] == brand: #다른가게 크롤방지
                        df = np.append(df, food_type)
                        df = np.append(df, brand)
                        df = np.append(df, countyName(" ".join(html.select('div.addr')[i].text.replace('\n','').split(' ')[0:2])))
        except:
            break
    
    df = pd.DataFrame(df.reshape(-1,3),columns =  [['category', 'brand', 'addr']]) #데이터 프레임화
    df.to_csv(f'{brand}.csv',encoding='utf-8-sig', index = False) #csv 저장


In [16]:
pizza_list = ['피자스쿨', '도미노피자', '피자헛', '피자알볼로', '피자에땅', '미스터피자', '반올림피자샵']

In [ ]:
for pizza in pizza_list:
    print(pizza, '크롤링 중')
    kakao_map_crawler('피자', pizza)
    print(pizza, '크롤링 완료')

피자스쿨 크롤링 중
